In [ ]:
from ultralytics import YOLO

# 베이스 모델 로드
model = YOLO("yolo11n.pt")

# 모델 학습
model.train(data='datasets/mask_ob/mask.yaml',
            epochs=20,
            project='mask',
            name='mask_model')

In [ ]:
import torch
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO("mask/mask_model/weights/best.pt")

# 평가 데이터 경로
root = Path("datasets/mask_ob/images/test")
output_dir = Path("datasets/mask_ob/cropped")  # 크롭된 이미지 저장 폴더
output_dir.mkdir(parents=True, exist_ok=True)  # 폴더가 없으면 생성

plt.figure(figsize=(20, 20))
i = 0

for path in root.glob("*.png"):
    # 모델 예측
    results = model(path)
    im_array = results[0].plot()  # 시각화용 이미지 배열
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL 이미지 변환

    # 원본 이미지 로드
    original_image = Image.open(path)

    # 예측된 객체별로 바운딩 박스 크롭 후 저장
    boxes = results[0].boxes.xyxy.cpu().numpy()  # 감지된 객체의 바운딩 박스 좌표 (x1, y1, x2, y2)

    for j, (x1, y1, x2, y2) in enumerate(boxes):
        cropped = original_image.crop((x1, y1, x2, y2))  # 객체 크롭
        cropped_path = output_dir / f"{path.stem}_object{j}.png"  # 저장 경로
        cropped.save(cropped_path)  # 크롭된 이미지 저장

    # 시각화
    i += 1
    ax = plt.subplot(3, 4, i)
    ax.imshow(im)
    ax.axis("off")  # 축 제거

plt.tight_layout()
plt.show()
